In [45]:
import re
import json
import string

import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import AffinityPropagation


In [46]:
def clean_string(string, regex=r'[^a-zA-Z\s0-9ÄÖÜäöü.?,;]+'):
    return re.sub(regex, '', string)

## read json data

In [74]:
with open('dienstleistungen.json') as data_file:    
    data = json.load(data_file)

In [95]:
bla = pd.DataFrame(data['data'])
bla.columns = ['appointment', 'authorities', 'description', 'fees', 'forms', 'id',
       'legal', 'Leika', 'links', 'locations', 'meta', 'name',
       'onlineprocessing', 'prerequisites', 'process_time', 'publications',
       'relation', 'representation', 'requirements', 'residence',
       'responsibility', 'responsibility_all']

In [48]:
def parse_names(data):
    names = []
    for entry in data['data']:
        if entry['name'] == tuple():
            tmp = clean_string(entry['name'][0])
            names.append(tmp)
        else:
            tmp = clean_string(entry['name'])
            names.append(tmp)
    return names

In [49]:
names = parse_names(data)

## grouping based on field 'leika'

In [50]:
#99144017215000
from collections import defaultdict

test = defaultdict(list)

for entry in data['data']:
    topic = entry['leika'][2:5]
    test[topic].append(entry)

In [53]:
for topic, data in test.items():
    print('topic', topic)
    print()
    topic_1 = data
    break
    for services in data:
        print(services['name'], services['leika'][8:11])
    print()

topic 106



In [55]:
for entry in topic_1:
    print(entry['leika'])

99106020000000
99106021000000
11106900000000


In [ ]:
## clustering

In [61]:
import pandas as pd

In [101]:
bla5 = pd.DataFrame.from_csv('data.tsv', sep='\t').reset_index()

In [104]:
bla5.columns = ['Leika', 'Leika Gruppe', 'Leika Kennung', 'Leika Verrichtung',
       'Leika Synonyme']

In [105]:
bla5.head()

,Leika,Leika Gruppe,Leika Kennung,Leika Verrichtung,Leika Synonyme
0,99001013006000,Abfall,Transport nach dem Kreislaufwirtschaftsgesetz,Genehmigung,Abfalltransport | Abfallsammler | Abfallbeförd...
1,99108008152000,Straßenverkehr,abgeschleppte Fahrzeuge,Herausgabe,Fahrzeug-Verwahrplatz | Autoknast | abgeschlep...
2,99115005070000,Wohnsitz,Wohnsitz,Abmeldung,Abmeldung | Wohnungsabmeldung | Wegzug | abmel...
3,99046058061000,Gerichtliche Leistungen,Abwesenheitspfleger,Bestellung,Pfleger für abwesende volljährige Personen | A...
4,99089070109000,Sicherheit und Ordnung,Akten im Bußgeldverfahren,Einsicht gewähren,Akteneinsicht | Fotoeinsicht | Beweismittelein...


In [114]:
result = pd.merge(bla, bla5, on='Leika', how='outer')

In [115]:
result.head()

,appointment,authorities,description,fees,forms,id,legal,Leika,links,locations,...,relation,representation,requirements,residence,responsibility,responsibility_all,Leika Gruppe,Leika Kennung,Leika Verrichtung,Leika Synonyme
0,{'link': 'https://service.berlin.de/terminvere...,"[{'id': '12667', 'name': 'Bezirksamt Charlotte...","<ul class=""list""><li>Sie geben eine Berliner W...",gebührenfrei; dies gilt auch für die Abmeldebe...,[{'link': 'http://www.berlin.de/formularserver...,120335,[{'link': 'http://www.gesetze-im-internet.de/b...,99115005070000,[{'link': 'https://www.bmi.bund.de/DE/Themen/M...,"[{'hint': '', 'url': '//service.berlin.de/dien...",...,{'root_topic': '324835'},0,"[{'link': '', 'name': 'Abmeldung bei der Melde...",0,Diese Dienstleistung erhalten Sie in allen Ber...,True,NaN,NaN,NaN,NaN
1,{'link': ''},"[{'id': '12667', 'name': 'Bezirksamt Charlotte...",Wohngeld kann Ihnen angemessenes und familien-...,keine,[{'link': 'https://senstadtfms.stadt-berlin.de...,120656,[{'link': 'http://www.gesetze-im-internet.de/b...,11107023017900,[{'link': 'http://www.bmub.bund.de/service/pub...,"[{'hint': '', 'url': '//service.berlin.de/dien...",...,{'root_topic': '324831'},0,[{'link': 'https://senstadtfms.stadt-berlin.de...,0,Den ausgefüllten Antrag auf Bewilligung von Wo...,false,NaN,NaN,NaN,NaN
2,{'link': 'https://service.berlin.de/terminvere...,"[{'id': '12667', 'name': 'Bezirksamt Charlotte...",Befindet sich die Hauptwohnung in Berlin und d...,"bis 11,00 Euro",[],120658,[{'link': 'http://www.gesetze-im-internet.de/s...,11036008014900,[],"[{'hint': '', 'url': '//service.berlin.de/dien...",...,{'root_topic': '324803'},0,"[{'link': '', 'name': 'Fahrzeugschein / Zulass...",0,Die Dienstleistung kann bei der Zulassungsbehö...,True,NaN,NaN,NaN,NaN
3,{'link': ''},"[{'id': '12667', 'name': 'Bezirksamt Charlotte...",Wohngeld kann Ihnen angemessenes und familien-...,keine,[{'link': 'https://senstadtfms.stadt-berlin.de...,120665,[{'link': 'http://www.gesetze-im-internet.de/b...,11107023017901,[{'link': 'http://www.bmub.bund.de/service/pub...,"[{'hint': '', 'url': '//service.berlin.de/dien...",...,{'root_topic': '324831'},0,[{'link': 'https://senstadtfms.stadt-berlin.de...,0,Den ausgefüllten Antrag auf Lastenzuschuss und...,false,NaN,NaN,NaN,NaN
4,{'link': ''},"[{'id': '12667', 'name': 'Bezirksamt Charlotte...",Mit einem Wohnberechtigungsschein (WBS) können...,Gebührenfrei,[{'link': 'https://senstadtfms.stadt-berlin.de...,120671,[{'link': 'http://www.gesetze-im-internet.de/b...,11107022900900,[],"[{'hint': '', 'url': '//service.berlin.de/dien...",...,{'root_topic': '324831'},0,[{'link': 'https://senstadtfms.stadt-berlin.de...,0,"Wohnungsamt des Bezirks, in dem Sie wohnen. Wo...",false,NaN,NaN,NaN,NaN


In [108]:
bla.head()

,appointment,authorities,description,fees,forms,id,legal,Leika,links,locations,...,onlineprocessing,prerequisites,process_time,publications,relation,representation,requirements,residence,responsibility,responsibility_all
0,{'link': 'https://service.berlin.de/terminvere...,"[{'id': '12667', 'name': 'Bezirksamt Charlotte...","<ul class=""list""><li>Sie geben eine Berliner W...",gebührenfrei; dies gilt auch für die Abmeldebe...,[{'link': 'http://www.berlin.de/formularserver...,120335,[{'link': 'http://www.gesetze-im-internet.de/b...,99115005070000,[{'link': 'https://www.bmi.bund.de/DE/Themen/M...,"[{'hint': '', 'url': '//service.berlin.de/dien...",...,"{'link': '', 'description': 'false'}","[{'link': '', 'name': 'keine', 'description': ...",Bei persönlicher Vorsprache in einem Bürgeramt...,[],{'root_topic': '324835'},0,"[{'link': '', 'name': 'Abmeldung bei der Melde...",0,Diese Dienstleistung erhalten Sie in allen Ber...,True
1,{'link': ''},"[{'id': '12667', 'name': 'Bezirksamt Charlotte...",Wohngeld kann Ihnen angemessenes und familien-...,keine,[{'link': 'https://senstadtfms.stadt-berlin.de...,120656,[{'link': 'http://www.gesetze-im-internet.de/b...,11107023017900,[{'link': 'http://www.bmub.bund.de/service/pub...,"[{'hint': '', 'url': '//service.berlin.de/dien...",...,"{'link': '', 'description': 'false'}",[{'link': 'http://www.stadtentwicklung.berlin....,false,[],{'root_topic': '324831'},0,[{'link': 'https://senstadtfms.stadt-berlin.de...,0,Den ausgefüllten Antrag auf Bewilligung von Wo...,false
2,{'link': 'https://service.berlin.de/terminvere...,"[{'id': '12667', 'name': 'Bezirksamt Charlotte...",Befindet sich die Hauptwohnung in Berlin und d...,"bis 11,00 Euro",[],120658,[{'link': 'http://www.gesetze-im-internet.de/s...,11036008014900,[],"[{'hint': '', 'url': '//service.berlin.de/dien...",...,"{'link': '', 'description': 'false'}","[{'link': '', 'name': 'Keine Voraussetzungen e...",false,[],{'root_topic': '324803'},0,"[{'link': '', 'name': 'Fahrzeugschein / Zulass...",0,Die Dienstleistung kann bei der Zulassungsbehö...,True
3,{'link': ''},"[{'id': '12667', 'name': 'Bezirksamt Charlotte...",Wohngeld kann Ihnen angemessenes und familien-...,keine,[{'link': 'https://senstadtfms.stadt-berlin.de...,120665,[{'link': 'http://www.gesetze-im-internet.de/b...,11107023017901,[{'link': 'http://www.bmub.bund.de/service/pub...,"[{'hint': '', 'url': '//service.berlin.de/dien...",...,"{'link': '', 'description': 'false'}",[{'link': 'http://www.stadtentwicklung.berlin....,false,[],{'root_topic': '324831'},0,[{'link': 'https://senstadtfms.stadt-berlin.de...,0,Den ausgefüllten Antrag auf Lastenzuschuss und...,false
4,{'link': ''},"[{'id': '12667', 'name': 'Bezirksamt Charlotte...",Mit einem Wohnberechtigungsschein (WBS) können...,Gebührenfrei,[{'link': 'https://senstadtfms.stadt-berlin.de...,120671,[{'link': 'http://www.gesetze-im-internet.de/b...,11107022900900,[],"[{'hint': '', 'url': '//service.berlin.de/dien...",...,"{'link': '', 'description': 'false'}","[{'link': '', 'name': 'Deutsche Staatsangehöri...",false,[],{'root_topic': '324831'},0,[{'link': 'https://senstadtfms.stadt-berlin.de...,0,"Wohnungsamt des Bezirks, in dem Sie wohnen. Wo...",false
